In [37]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import shutil
import os
import IPython.display
import random
import math
import shutil
import pandas as pd

In [2]:
def load(file_name):
    wav = file_name
    file_sr = librosa.get_samplerate(wav)

    y, sr = librosa.load(wav, sr=file_sr)
    return y, sr

In [3]:
def decibel_normalization(input_directory, output_directory, decibel):
    file_list = os.listdir(input_directory)
    
    for file in file_list:
        new_audio = []
        y, sr = load(input_directory+file)
        max_mine = np.max(y)
        ratio = decibel / max_mine
        for i in range(len(y)):
            new_audio.append(y[i] * ratio)
        sf.write(output_directory+file, new_audio, sr, subtype='PCM_16')   

In [6]:
decibel_normalization('1202/성별/남자/', '1202/normalization/남자/', 0.9999695)

In [7]:
decibel_normalization('1202/성별/여자/', '1202/normalization/여자/', 0.9999695)

In [32]:
dir1 = '1202/normalization/남자/'
dir2 = '1202/pitch/남자/'
files = os.listdir(dir1)
for f in files:
    y, sr = load(dir1+f)
    new_audio = librosa.effects.pitch_shift(y,sr,n_steps=3)

    i = 4
    if(f.split('_')[0] == '장석영'):
        i = 5
        
    sf.write(dir2 + f.split('_')[0] + '_'+str(int(f.split('_')[1][:-4])+i) + ".wav", new_audio, sr, subtype='PCM_16')

In [31]:
dir1 = '1202/normalization/여자/'
dir2 = '1202/pitch/여자/'
files = os.listdir(dir1)
for f in files:
    y, sr = load(dir1+f)
    new_audio = librosa.effects.pitch_shift(y,sr,n_steps=-3)

    sf.write(dir2 + f.split('_')[0] + '_'+str(int(f.split('_')[1][:-4])+4) + ".wav", new_audio, sr, subtype='PCM_16')

In [33]:
y, sr = load('1202/normalization/장석영_5.wav')
new_audio = librosa.effects.pitch_shift(y,sr,n_steps=3)
sf.write('1202/normalization/장석영_10.wav', new_audio, sr, subtype='PCM_16')

In [28]:
a = '장석영'
i = 1
if(a == '장석영'):
    i = 5
print(i)

5


In [39]:
def add_noise(input_directory, output_directory, percent):
    file_list = os.listdir(input_directory)
    
    for file in file_list:
        y, sr = load(input_directory+file)
        #random_state = 42
        #rnd_state = np.random.RandomState(random_state)
        
        data = np.c_[y]
        mu = np.mean(data)
        sd = np.std(data)
        
        data_df = pd.DataFrame(data, columns=['Value'])
        data_df['Index'] = data_df.index.values
        
        ratio = np.random.uniform(0.05, percent)
        
        jitter = ratio*np.random.normal(mu, sd, size=data_df.shape[0])
        data_df['with_jitter'] = data_df['Value'] + jitter
        
        sf.write(output_directory + 'noise_' + file, data_df['with_jitter'], sr, subtype='PCM_16')

In [40]:
add_noise('1202/합본/', '1202/노이즈/', 0.1)